In [4]:
import tensorflow as tf
from tensorflow import layers

tf.logging.set_verbosity(tf.logging.INFO)

In [8]:
def model(features, labels, mode):
    input_layer = features['x']
    
    conv1 = layers.conv2d(inputs=input_layer,
                         filters=64,
                         kernel_size=(1,3),
                         padding='valid',
                         )
    
    conv2 = layers.conv2d(inputs=conv1,
                         filters=64,
                         kernel_size=(1,1),
                         padding='valid',
                         )
    
    conv3 = layers.conv2d(inputs=conv2,
                         filters=64,
                         kernel_size=(1,1),
                         padding='valid'
                         )
    
    conv4 = layers.conv2d(inputs=conv3,
                         filters=128,
                         kernel_size=(1,1),
                         padding='valid'
                         )
    
    conv5 = layers.conv2d(inputs=conv4,
                         filters=1024,
                         kernel_size=(1,1),
                         padding='valid'
                         )
    
    
    max1 = layers.max_pooling2d(inputs=conv5,
                                pool_size=(n,1)
                               )
    
    
    reshape1 = layers.flatten(inputs=max1) #Test
    
    
    dense1 = layers.dense(inputs=reshape1,units=512)
    
    dense2 = layers.dense(inputs=reshape2,units=256)
    
    dense3 = layers.dense(inputs=reshape3,units=128)
    
    expand = tf.tile(tf.reshape(dense3,[1,1,1,-1]),[1,n,1,1])
    
    concat = tf.concat(axis=3, values=[dense3, expand])
    
    conv6 = layers.conv2d(inputs=concat,
                         filters=512,
                         kernel_size=(1,1),
                         padding="valid"
                         )
    
    conv7 = layers.conv2d(inputs=conv6,
                         filters=256,
                         kernel_size=(1,1),
                         padding='valid'
                         )
    drop = layers.dropout(inputs=conv7,
                           rate=0.7)
    
    conv8 = layers.conv2d(inputs=drop,
                          filters=17,
                          kernel_size=(1,1),
                          padding='valid'
                         )
    
    logits = tf.squeeze(conv8,[2],name="logits")
    
    prediction = {
        "classes" : tf.arg_max(logits,axis=1),
        "probabilities" : logits
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    
    
        

In [9]:
# Create the estimator
classifier = tf.estimator.Estimator(model_fn=model, model_dir='./temp/pointNet/')
print(classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_summary_steps': 100, '_service': None, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd779ba2fd0>, '_evaluation_master': '', '_session_config': None, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_steps': None, '_train_distribute': None, '_log_step_count_steps': 100, '_model_dir': './temp/pointNet/', '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker'}


In [10]:
# Log
tensors_to_log = {"probabilities": "logits"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10000)

In [ ]:
# Training 
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": np.asarray(pointCld, dtype=np.float32)},
y=train_labels, batch_size=1, num_epochs=None, shuffle=True)